In [1]:
%display latex

# Gaussian propagation through RL iterations

This is a *SageMath* notebook!
https://www.sagemath.org/index.html

If in windows, you need to run from WSL + Ubuntu/Debian + miniconda environment containing `sage` package. Don't try to run this in VS code.

In linux, install miniconda (new environment) and install `sage` package from conda-forge, for example `conda create -n sage_env python=3.8 sage -c conda-forge`.

To start this SageMath notebook, run from the *activated* command line `sage -n jupyter` and navigate to this file.

This notebook examines the effect that RL deconvolution has with 1D gaussian functions for psf and data.

## 'Gaussian' mathematics

There are important properties of gaussian functions centered at zero that are exploited in this analysis.

Consider the generic Gaussian function (centered at zero)

In [2]:
G(x,sigma,A) = A*exp(-x^2/sigma^2)
G(x,sigma,A)

A*e^(-x^2/sigma^2)

Then the...

**1. Multiplication (or division) of two gaussians is another gaussian**

In [3]:
x, sigma1, sigma2, A1, A2, t = var('x, sigma1, sigma2, A1, A2, t')
expr= G(x,sigma1,A1)*G(x,sigma2,A2)
expr

A1*A2*e^(-x^2/sigma1^2 - x^2/sigma2^2)

with the new $\sigma_{3}$ given by

In [4]:
sigma3 = -2/log(expr).diff(x,2)
sigma3.simplify()

1/(1/sigma1^2 + 1/sigma2^2)

**2. Convolution of two gaussians is another gaussian**

In [5]:
expr= integrate( G(t,sigma1,A1)*G(t-x,sigma2,A2) , t, -infinity, infinity,  assume(sigma1^2+sigma2^2>0))
expr

sqrt(pi)*A1*A2*e^(-x^2/(sigma1^2 + sigma2^2))/sqrt((sigma1^2 + sigma2^2)/(sigma1^2*sigma2^2))

with the new $\sigma_{3}$ given by

In [6]:
sigma3 = -2/log(expr).diff(x,2)
sigma3.simplify()

sigma1^2 + sigma2^2

## Richardson-Lucy iterations

Consider the following data. A single point (dirac delta) that has been convoluted with a PSF. This results in the 'Experimental Image' that is basically identical to the PSF, so it is a gaussian with the same _sigma_(standard deviation).

1. Run RL iteration
2. Check resulting gaussian (fit peak or calculate), and collect new _sigma_
3. back to 1

How the _sigma_ value varies with iteration?

This can be calculated analytically

PSF is the $h(x)$ function.  PSF sigma's is in the $b^{2}= \sigma^{2}$ value.

Assume to start that PSF is the starting image (iteration zero)

We focus our attention to the resulting denominator term inside the exponential under x^2

In [7]:
b,h,t,x, H= var('b h t x H')

In [11]:
h(x) = exp(-x^2/b^2)

In [12]:
# rl_1 = term4 = h(x) x ( h(-x) * (h(x) / (h(x) * h(x) )) ) 
# term3 = h(x) x term2
# term2 = h(-x) * term1
# term1 = h(x) / term0
# term0 = h(x) * h(x)

In [13]:
# Convolution h(x) * h(x)
term0(x) = integrate( h(t)*h(t-x) , t, -infinity, infinity)
term0

x |--> 1/2*sqrt(2)*sqrt(pi)*sqrt(b^2)*e^(-1/2*x^2/b^2)

In [14]:
term1(x) = h(x) / term0(x)
term1

x |--> sqrt(2)*e^(-1/2*x^2/b^2)/(sqrt(pi)*sqrt(b^2))

In [15]:
term2(x) = integrate( h(-t)*term1(t-x) , t, -infinity, infinity)
term2

x |--> 2/3*sqrt(2)*sqrt(3/2)*e^(-1/3*x^2/b^2)

In [16]:
term3 = h(x)*term2(x)
term3

2/3*sqrt(2)*sqrt(3/2)*e^(-4/3*x^2/b^2)

In [17]:
res(x) = simplify(term3)
res(x)

2/3*sqrt(3)*e^(-4/3*x^2/b^2)

The result after first iteration is another gaussian but with different variance

try to extract term under denominator

In [18]:
expand(ln(res(x)))

log(2/3*sqrt(3)*e^(-4/3*x^2/b^2))

In [16]:
f(x) = ln(res(x))

In [17]:
-2/f.diff(x,2)

x |--> 3/4*b^2

Note the denominator term inside the exponential went from $b^2$ to $3/4b^2$, smaller width

Now, what happens if the later iterations, where the earlier iteration is a gaussian with a different variance than PSF. Note that the starting image (iteration=0) is still assumed to be equal to the PSF

In [18]:
R = var('R')
r = var('r')

Iteration

In [19]:
Rn(x) = R*exp(-x^2/r^2)

variance of Rn is $r^2$

PSF

In [20]:
h(x) = H*exp(-x^2/b^2)

In [21]:
# rl_n = rterm3
# rterm3 = Rn(x) x rterm2
# rterm2 = h(-x) * rterm1
# rterm1 = h(x) / rterm0  #R0=h(x) in this example
# rterm0 = h(x) * Rn(x)

In [22]:
#Convolution at denominator
rterm0(x) = integrate( h(t)*Rn(t-x) , t, -infinity, infinity, assume(r^2+b^2>0))

In [23]:
rterm1(x) = h(x) / rterm0(x)

In [24]:
#Convolution with flipped PSF
rterm2(x) = integrate( h(-t)*rterm1(t-x) , t, -infinity, infinity, assume(2*r^2+b^2>0))

In [25]:
rterm3(x) = Rn(x) * rterm2(x)

In [26]:
simplify(rterm3)

x |--> sqrt((b^2 + r^2)/(b^2*r^2))*e^(-r^2*x^2/(b^4 + 2*b^2*r^2) - x^2/r^2)/sqrt((b^2 + 2*r^2)/(b^4 + b^2*r^2))

Interesting enough, The $R$ amplitude in $R_n$ is not appearing in this result

Collect term under $x^2$ using the log diff-trick

In [27]:
f(x) = ln(rterm3(x))

In [28]:
simplify(-2/f.diff(x,2))

x |--> 1/(r^2/(b^4 + 2*b^2*r^2) + 1/r^2)

This is the new variance value as a function of the original PSF variance $b^2$ and from previous iteration $r^2$

Construct a new function that calculates variance from previous iterations variances

In [29]:
nextiter(b,r) = simplify(-2/f.diff(x,2))
nextiter

(b, r) |--> 1/(r^2/(b^4 + 2*b^2*r^2) + 1/r^2)

In [30]:
#iteration zero
iter0(b)= nextiter(b,b) 
iter0

b |--> 3/4*b^2

In [31]:
iter1(b) = nextiter(b, iter0(b))
simplify(iter1)

b |--> 18/(81*b^4/(9*b^6 + 8*b^4) + 32/b^4)

quite complex

See what happens if we start with PSF variance b=1

In [32]:
iter0=nextiter(1,1)
N(iter0)

0.750000000000000

In [33]:
iter1 = nextiter(1,iter0)
N(iter1)

0.489600000000000

In [34]:
iter2 = nextiter(1,iter1)
N(iter2)

0.230746065165781

In [35]:
iter3 = nextiter(1,iter2)
N(iter3)

0.0531076810823308

In [36]:
iter4 = nextiter(1,iter3)
N(iter4)

0.00282040348003919

Value for the gaussian variance decreases for each iteration